In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt

class Discriminator(nn.Module):
    def __init__(self,img_dim):
        super().__init__()
        self.disc = nn.Sequential(
            nn.Linear(img_dim,128),
            nn.LeakyReLU(0.1),
            nn.Linear(128,1),
            nn.Sigmoid()
             )
    def forward(self,x):
        return self.disc(x)

class Generator(nn.Module):
    def __init__(self,z_dim,img_dim):
        super().__init__()
        self.gen = nn.Sequential(
            nn.Linear(z_dim,256),
            nn.LeakyReLU(0.1),
            nn.Linear(256,img_dim),
            nn.Tanh()
              )

    def forward(self,x):
         return self.gen(x)

device = "cuda" if torch.cuda.is_available() else "cpu"
lr = 5e-4
z_dim = 64
image_dim = 28 * 28 * 1
batch_size = 32
num_epochs = 50

disc = Discriminator(image_dim).to(device)
gen = Generator(z_dim,image_dim).to(device)
fixed_noise = torch.randn(batch_size,z_dim).to(device)
transform = transforms.Compose(
[transforms.ToTensor(),transforms.Normalize((0.5,),(0.5,))]
)
dataset = datasets.MNIST(root = "dataset/", transform = transform,download=True)
loader = DataLoader(dataset, batch_size=batch_size,shuffle=True)
opt_disc = optim.Adam(disc.parameters(),lr =lr)
opt_gen = optim.Adam(gen.parameters(),lr = lr)
criterion = nn.BCELoss()
writer_fake = SummaryWriter(f"runs/GAN_MNIST/fake")
writer_real = SummaryWriter(f"runs/GAN_MNIST/real")
step = 0

for epoch in range(num_epochs):
    for batch_idx, (real,_) in enumerate(loader):
            real = real.view(-1,784).to(device)
            batch_size = real.shape[0]

            noise = torch.randn((batch_size,z_dim)).to(device)
            fake = gen(noise)
            disc_real = disc(real).view(-1)
            lossD_real = criterion(disc_real,torch.ones_like(disc_real))
            disc_fake = disc(fake).view(-1)
            lossD_fake = criterion(disc_fake,torch.zeros_like(disc_fake))
            lossD = (lossD_fake + lossD_real)/2
            disc.zero_grad()
            lossD.backward(retain_graph = True)

            opt_disc.step()

            output = disc(fake).view(-1)
            lossG = criterion(output,torch.ones_like(output))
            gen.zero_grad()
            lossG.backward()
            opt_gen.step()

            if batch_idx == 0:
                        print(
                            f"Epoch [{epoch}/{num_epochs}] Batch {batch_idx}/{len(loader)} \
                                  Loss D: {lossD:.4f}, loss G: {lossG:.4f}"
                        )

                        with torch.no_grad():
                            fake = gen(fixed_noise).reshape(-1, 1, 28, 28)
                            data = real.reshape(-1, 1, 28, 28)

                            img_grid_fake = torchvision.utils.make_grid(fake, normalize=True)
                            img_grid_real = torchvision.utils.make_grid(data, normalize=True)

                            # print(type(img_grid_fake.cpu().detach().numpy()))
                            # writer_fake.add_image(
                            #     "Mnist Fake Images", img_grid_fake, global_step=step
                            # )
                            # writer_real.add_image(
                            #     "Mnist Real Images", img_grid_real, global_step=step
                            # )
                            plt.imshow(img_grid_fake.cpu().permute(1, 2, 0))


                            step += 1

C:\ProgramData\Anaconda3\envs\My_env\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'C:\ProgramData\Anaconda3\envs\My_Env\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


Epoch [0/50] Batch 0/1875                                   Loss D: 0.7240, loss G: 0.7393
Epoch [1/50] Batch 0/1875                                   Loss D: 0.7213, loss G: 0.9795
Epoch [2/50] Batch 0/1875                                   Loss D: 0.3027, loss G: 1.5648
Epoch [3/50] Batch 0/1875                                   Loss D: 0.5410, loss G: 0.9576
Epoch [4/50] Batch 0/1875                                   Loss D: 0.8662, loss G: 0.7797
Epoch [5/50] Batch 0/1875                                   Loss D: 0.9825, loss G: 0.7609
Epoch [6/50] Batch 0/1875                                   Loss D: 0.5725, loss G: 1.2303
Epoch [7/50] Batch 0/1875                                   Loss D: 0.6360, loss G: 1.5735
Epoch [8/50] Batch 0/1875                                   Loss D: 0.6630, loss G: 0.9989
Epoch [9/50] Batch 0/1875                                   Loss D: 0.7848, loss G: 0.8488
Epoch [10/50] Batch 0/1875                                   Loss D: 0.6747, loss G: 1.059